In [1]:
   %load_ext autoreload
   %autoreload 2

In [ ]:

from ErrP_decoder.data_io import select_eeg_files, load_selected_files

file_paths = select_eeg_files()

datasets = load_selected_files(file_paths)

for ds in datasets:
    print("Data shape:", ds.data.shape)
    print("Info:", ds.info)

In [3]:
from ErrP_decoder.data_io import set_params
for ds in datasets:
    cfg = set_params(ds.info,config_path="../configs/default.yaml")

In [ ]:
from ErrP_decoder.data_io import preprocess_dataset
for ds in datasets:
    ds = preprocess_dataset(ds, cfg)

for ds in datasets:
    neu = sum(ds.trig['typ'] == 0)
    pos = sum(ds.trig['typ'] == 1)
    neg = sum(ds.trig['typ'] == 2)
    print(neu)
    print(pos)
    print(neg)
    


In [ ]:
from ErrP_decoder.data_io import bandpass_filter
from ErrP_decoder.plots import plot_eeg_interactive

for ds in datasets:
    filtered_eeg = bandpass_filter(ds.eeg,cfg)
    plot_eeg_interactive(filtered_eeg, cfg)
    ds.eeg_filt = filtered_eeg

In [6]:
from ErrP_decoder.data_io import epoch_data
for ds in datasets:
    ds.epochs, ds.epoch_labels = epoch_data(ds.eeg_filt,cfg,ds.trig)
    # epochs (768,32,200)
    # epoch_labels (200,)

In [7]:
import numpy as np

all_epochs = []
all_labels = []
for ds in datasets:
    all_epochs.append(ds.epochs)
    all_labels.append(ds.epoch_labels)
# Concatenate along the epochs axis (axis=2 for epochs, axis=0 for labels)
epochs_combined = np.concatenate(all_epochs, axis=2)
labels_combined = np.concatenate(all_labels, axis=0)

In [ ]:
from ErrP_decoder.plots import plot_erp
plot_erp(epochs_combined, labels_combined, cfg, 'CZ')
plot_erp(epochs_combined, labels_combined, cfg, 'FCZ')
plot_erp(epochs_combined, labels_combined, cfg, 'FZ')
plot_erp(epochs_combined, labels_combined, cfg, 'PZ')
plot_erp(epochs_combined, labels_combined, cfg, 'CPZ')

In [9]:
from ErrP_decoder.data_io import balance_runs
for ds in datasets:
    ds.epochs_balanced, ds.epoch_labels_balanced = balance_runs(ds.epochs,ds.epoch_labels)

In [11]:
for ds in datasets:
    print(ds.epoch_labels_balanced.shape)


(50,)
(66,)
(80,)
(58,)
(52,)


In [18]:

all_epochs = []
all_labels = []
run_labels = []

for i, ds in enumerate(datasets):
    all_epochs.append(ds.epochs_balanced)              # shape: (samples, channels, trials)
    all_labels.append(ds.epoch_labels_balanced)        # shape: (trials,)

    n_trials = ds.epoch_labels_balanced.shape[0]
    run_labels.append(np.full(n_trials, i))            # start from 0

# Concatenate everything
training_data   = np.concatenate(all_epochs, axis=2)   # shape: (768,32,306)
training_labels = np.concatenate(all_labels, axis=0)   # shape: (306,)
run_labels      = np.concatenate(run_labels, axis=0)   # shape: (306,)



In [20]:
from ErrP_decoder.modeling import get_cca_spatialfilter

spatial_filter = get_cca_spatialfilter(training_data, training_labels, 
                                       n_components=cfg['spatial_filter']['n_comp']) # shape (32,2)



In [ ]:

from ErrP_decoder.modeling import apply_spatial_filter
filtered_data = apply_spatial_filter(training_data, spatial_filter) # shape: (samples, n_components, trials)